In [540]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm import tqdm_notebook
import concurrent.futures

In [3]:
import pandas as pd
import requests
import time
from collections import deque
from lxml import etree
import sys
from tqdm import tqdm_notebook

# 获取词条的WikiURL

In [559]:
import requests
import json
from bs4 import BeautifulSoup
def get_url(word):
    res = requests.get("https://en.wikipedia.org/wiki/%s"%word)
    content = BeautifulSoup(res.text)
    wikiTitle = content.findAll("link",rel="canonical")[0].attrs['href'].split("/")[-1]
    return [str(res.status_code),wikiTitle]

with open("/home/zhoutong/tags/tagMap_9k.txt") as f:
    content = [i.strip("\n").split("\t") for i in f.readlines()]
    print(len(content))
# content[:3]
#     get_url("Pregnancy Photography")

10857


In [567]:

with open("/home/zhoutong/tags/tagMap_9k.json"+par,"w+") as f:
    for (tag, checkList) in tqdm_notebook(content):
        tagInfo = ",".join([tag] + get_url(tag))
        checkInfo = "|".join([",".join([i] + get_url(i)) for i in checkList.split(",")])
#         tagInfo+"\t"+checkInfo
        f.writelines(tagInfo+"\t"+checkInfo+"\n")

## 后台文件切块后开多个pyhton进程处理

In [581]:
par="_par0%s" % 3
print("processing %s" % par)

with open("/home/zhoutong/tags/tagMap_9k.txt"+par,"r+") as f:
    content = [i.strip().split("\t") for i in f.readlines()]
    
    content[85:90]

processing _par03


[['Gopichand or Gopiyantra or Khamak', 'Gopichand or Gopiyantra or Khamak'],
 ['Gottuvadhyam or Chitravina', ' Gottuvadhyam or Chitravina'],
 ['Sarod', 'Sarod'],
 ['Sitar', 'Sitar'],
 ['Surbahar', 'Surbahar']]

In [546]:
import requests
import json
import sys
from bs4 import BeautifulSoup
from tqdm import tqdm
def get_url(word):
    res = requests.get("https://en.wikipedia.org/wiki/%s"%word)
    content = BeautifulSoup(res.text,"lxml")
    wikiTitle = content.findAll("link",rel="canonical")[0].attrs['href'].split("/")[-1]
    return [str(res.status_code),wikiTitle]

par="_par0%s" % sys.argv[1]
print("processing %s" % par)
with open("/home/zhoutong/tags/tagMap_9k.txt"+par,"r+") as f:
    content = [i.strip().split("\t") for i in f.readlines()]
    content = [list(filter(lambda x: x!="",i)) for i in content]

print("conetnt size is " + str(len(content)))
with open("/home/zhoutong/tags/tagMap_9k.json"+par,"w+") as f:
    for (tag, checkList) in tqdm(content):
        tagInfo = ",".join([tag] + get_url(tag))
        checkInfo = "|".join([",".join([i] + get_url(i)) for i in checkList.split(",")])
#         tagInfo+"\t"+checkInfo
        f.writelines(tagInfo+"\t"+checkInfo+"\n")

3000


## 尝试多线程

In [543]:
pbar = tqdm_notebook(total=len(content))
with open("/home/zhoutong/tags/tagMap_9k.json","w+") as f:
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        def run(tag,checkList,theFile,bar):
            tmpDict = {"tag":tag,"url":get_url(tag)}
            tmpDict.update({"toCheck":[{"word":i,"url":get_url(i)} for i in checkList.split(",")]})
            theFile.writelines(json.dumps(tmpDict)+"\n")
            bar.update(1)
        executor.map(lambda x,y : run(x,y,f,pbar), content)
pbar.close()

Traceback (most recent call last):
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <function <lambda> at 0x7fe8b9300048>: attribute lookup <lambda> on __main__ failed
Traceback (most recent call last):
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/reduction.py", line 51, in dumps
    cls(buf, protocol).dump(obj)
_pickle.PicklingError: Can't pickle <function <lambda> at 0x7fe8b9300048>: attribute lookup <lambda> on __main__ failed
Traceback (most recent call last):
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/queues.py", line 234, in _feed
    obj = _ForkingPickler.dumps(obj)
  File "/home/

<generator object _chain_from_iterable_of_lists at 0x7fe8b8adf990>

Process Process-8:
Process Process-6:
Process Process-7:
Process Process-5:
Traceback (most recent call last):


KeyboardInterrupt: 

  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhoutong/python3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zhoutong/python3/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/zhoutong/python3/lib/python3.6/concurrent/futures/process.py", line 169, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/zhouton

# wikipedia API使用

In [2]:
import wikipedia
# from wikipedia import PageError,DisambiguationError

## 提取词语的category
- 目的是评价词条是不是印度词汇，如果是印度词汇则记录下来后续对idf加倍

### 判断是否是印度相关词
- 包含词有  india indian hindu

In [12]:
def containsIndia(inp):
    return any(i in inp.lower() for i in ["india","indian","hindu"])
containsIndia("1980 establishments in India")

True

### 测试用例

In [32]:
word = "TMC"
try:
    page = wikipedia.page(word)
    page.original_title
    any(containsIndia(i) for i in page.categories)
    page.categories
except wikipedia.PageError as e:
    print("PageError on %s" % word)
except wikipedia.DisambiguationError as e:
    print("DisambiguationError on %s" % word)


DisambiguationError on TMC


### 读取文件批量处理

In [31]:
with open("/home/zhoutong/tags/toCheckIsIndiaAndUpgradeIDF.txt","r+") as f:
    words = [i.strip("\n") for i in f.readlines()]
result = []
with open("/home/zhoutong/tags/toCheckIsIndiaAndUpgradeIDF_result.txt","w+") as f:
    for w in tqdm_notebook(words):
        if containsIndia(w):
            print("contains india in word: "+w)
        else:
            try:
                page = wikipedia.page(w)
                ori_title = page.original_title
                categories = page.categories
                if any(containsIndia(cat) for cat in categories):
                    result.append(w)
                f.writelines(w+"\n")
            except wikipedia.PageError as e:
                pass
    #             print("PageError on %s" % w)
            except wikipedia.DisambiguationError as e:
                pass
    #             print("DisambiguationError on %s" % w)

    

contains india in word: Indian
contains india in word: Hindu
contains india in word: thehindu
contains india in word: Indians
contains india in word: timesofindia
contains india in word: Indian_Express_App
contains india in word: Reserve_Bank_of_India
contains india in word: Indian_Express_App_Read
contains india in word: Indian_Express
contains india in word: Indian_Premier_League
contains india in word: indiatimes
contains india in word: India_News
contains india in word: Hans_India_News_App
contains india in word: Hindus
contains india in word: Indian_Penal_Code
contains india in word: Indian_Army
contains india in word: Hindustan_Times
contains india in word: State_Bank_of_India
contains india in word: Mumbai_Indians
contains india in word: Government_of_India
contains india in word: Hindu_Business_Line
contains india in word: Air_India
contains india in word: Hindutva
contains india in word: Team_India
contains india in word: OneIndia_News
contains india in word: india
contains in

# RNN

## 用RNN实现一个八位的二进制加法运算
- 参考： http://iamtrask.github.io/2015/11/15/anyone-can-code-lstm/ 

In [5]:
import numpy as np
np.random.seed(0)

### 基本函数定义

In [ ]:
# 激活函数用sigmoid
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output
# sigmoid导数
def sigmoid_derivation(output):
    return output*(1-output)

### 训练集生成

In [27]:
# 训练集
int2binary={}
binary_dim=8 # 8位二进制
largest_number = pow(2,binary_dim) # 8位二进制最大为2^8=256
# 使用np的API直接将整型转换为8位二进制
binary = np.unpackbits(np.array([range(largest_number)], dtype=np.uint8).T,axis=1)
for i in range(largest_number):
    int2binary.update({i:binary[i]})
int2binary[2]
int2binary[128]

array([0, 0, 0, 0, 0, 0, 1, 0], dtype=uint8)

array([1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

### 超参数

In [28]:
alpha=0.1
input_dim=2
hidden_dim=16
output_dim=1

### 初始化权重